In [1]:
from data import load_data
import matplotlib.pyplot as plt
import os
import numpy as np

import time

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

from keras.models import Model
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.layers.recurrent import GRU
from keras.layers import Input
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding3D
from keras.layers.core import Lambda, Dropout, Flatten, Dense, Activation
from keras.optimizers import Adam
from keras import backend as K

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
Using TensorFlow backend.


In [2]:
CURRENT_PATH = '/home/ubuntu/assignments/machine-lip-reading/preprocessing'
DATA_PATH = CURRENT_PATH + '/../data'

In [3]:
def ctc_lambda_func(args):
    import tensorflow as tf
    y_pred, labels, input_length, label_length = args
    # From Keras example image_ocr.py:
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    # y_pred = y_pred[:, 2:, :]
    label_length = K.cast(tf.squeeze(label_length),'int32')
    input_length = K.cast(tf.squeeze(input_length),'int32')
    labels = K.ctc_label_dense_to_sparse(labels, label_length)
    #y_pred = y_pred[:, :, :]
    #return K.ctc_batch_cost(labels, y_pred, input_length, label_length, ignore_longer_outputs_than_inputs=True)
    return tf.nn.ctc_loss(labels, y_pred, input_length, ctc_merge_repeated=False,
                         ignore_longer_outputs_than_inputs = True, time_major = False)

In [4]:
def CTC(name, args):
	return Lambda(ctc_lambda_func, output_shape=(1,), name=name)(args)

In [5]:
def build_model(input_size, output_size = 28, max_string_len = 10):
    # model = Sequential()
    input_data = Input(name='the_input', shape=input_size, dtype='float32')
    x = ZeroPadding3D(padding=(0,2,2), name='padding1')(input_data)
    x = TimeDistributed(Conv2D(filters = 32, kernel_size = 5, strides = (2,2),
                             padding = 'same', activation = 'relu'))(x)
    print
    x = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, name='max1'))(x)
    x = Dropout(0.5)(x)

    x = TimeDistributed(Conv2D(filters=32, kernel_size=5, strides=(2, 2),
                               padding='same', activation='relu'))(x)
    x = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, name='max1'))(x)
    x = Dropout(0.5)(x)

    x = TimeDistributed(Conv2D(filters=4, kernel_size=5, strides=(2, 2),
                               padding='same', activation='relu'))(x)
    x = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, name='max1'))(x)
    x = Dropout(0.5)(x)

    input_lstm = TimeDistributed(Flatten())(x)

    x_lstm = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru1'), merge_mode='concat')(input_lstm)
    x_lstm = Dense(output_size, kernel_initializer='he_normal', name='dense1')(x_lstm)
    print("after dense1")
    y_pred = Activation('softmax', name='softmax')(x_lstm)

    labels = Input(name='the_labels', shape = [max_string_len], dtype='int32')
    input_length = Input(name = 'input_length', shape =[1], dtype = 'int32')
    label_length = Input(name = 'label_length', shape = [1], dtype = 'int32')
    loss = CTC('ctc',[y_pred, labels, input_length, label_length])
    model = Model(inputs=[input_data, labels, label_length, input_length],
                  outputs = loss)
    model.summary()
    # Build model here...

    return model

In [6]:
def pad_labels(labels, max_string_len):
    padding = np.ones((labels.shape[0], max_string_len - labels.shape[1])) * -1
    return np.concatenate((labels, padding), axis = 1)

In [7]:
def train(model, x_train, y_train, label_len_train, input_len_train, batch_size=256, epochs=100, val_train_ratio=0.2):
    max_string_len = 10
    if y_train.shape[1] != max_string_len:
        y_train = pad_labels(y_train, max_string_len)

    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam)
    history = model.fit(x = {'the_input':x_train, 'the_labels':y_train, 'label_length':label_len_train,
                             'input_length':input_len_train}, y = {'ctc': np.zeros([x_train.shape[0]])},
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_split=val_train_ratio,
                        shuffle=True,
                        verbose=1)

    return history

In [8]:
def read_data():
    oh = OneHotEncoder()
    le = LabelEncoder()

    x = list()
    y = list()
    t = list()
    print("loading images...")
    for i, (img, words) in enumerate(load_data(DATA_PATH, verbose=False, framebyframe=False)):
        if img.shape[0] != 75:
            continue
        x.append(img)
        y.append(words)

        t += words.tolist()
        if i == 3:
            break

    t = le.fit_transform(t)
    oh.fit(t.reshape(-1, 1))

    print("convering to np array...")
    x = np.stack(x, axis=0)

    print("transforming y...")
    for i in range(len(y)):
        y_ = le.transform(y[i])
        y[i] = np.asarray(oh.transform(y_.reshape(-1, 1)).todense())
    y = np.stack(y, axis=0)

    return x, y

In [10]:
from data import read_data_for_speaker
speaker_id = 's13'
if speaker_id:
    start = time.time()
    
    x_raw, y_raw, word_len_list, input_len_list = read_data_for_speaker('s13')
    end = time.time()
    print("load data took", end-start)
    print("training data shapes:", x_raw.shape, y_raw.shape)
    x_train, x_test, y_train, y_test, label_len_train, label_len_test, \
    input_len_train, input_len_test = train_test_split(x_raw, y_raw, word_len_list, input_len_list, test_size=0.2)

    model = build_model(x_raw.shape[1:], 28, max_string_len = 10)

    history = train(model, x_train, y_train, label_len_train, input_len_train, epochs=100)

    print("Saving model...")
    model.save('model_'+ speaker_id +".h5")

    print("Done.")

('load data took', 0.0897209644317627)
('training data shapes:', (60, 16, 50, 100, 3), (60, 6))

after dense1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 16, 50, 100,  0                                            
__________________________________________________________________________________________________
padding1 (ZeroPadding3D)        (None, 16, 54, 104,  0           the_input[0][0]                  
__________________________________________________________________________________________________
time_distributed_8 (TimeDistrib (None, 16, 27, 52, 3 2432        padding1[0][0]                   
__________________________________________________________________________________________________
time_distributed_9 (TimeDistrib (None, 16, 13, 26, 3 0           time_distributed_8[0][0]         

Epoch 36/100
38/38 [==============================] - 2s 40ms/step - loss: 16.4847 - val_loss: 16.1308
Epoch 37/100
38/38 [==============================] - 1s 39ms/step - loss: 16.3456 - val_loss: 16.0167
Epoch 38/100
38/38 [==============================] - 2s 40ms/step - loss: 16.2614 - val_loss: 15.9065
Epoch 39/100
38/38 [==============================] - 2s 40ms/step - loss: 16.1120 - val_loss: 15.8066
Epoch 40/100
38/38 [==============================] - 1s 39ms/step - loss: 16.0726 - val_loss: 15.7128
Epoch 41/100
38/38 [==============================] - 1s 39ms/step - loss: 16.0161 - val_loss: 15.6256
Epoch 42/100
38/38 [==============================] - 1s 39ms/step - loss: 16.0013 - val_loss: 15.5447
Epoch 43/100
38/38 [==============================] - 2s 40ms/step - loss: 15.7949 - val_loss: 15.4765
Epoch 44/100
38/38 [==============================] - 1s 39ms/step - loss: 15.7522 - val_loss: 15.4141
Epoch 45/100
38/38 [==============================] - 2s 40ms/step - loss

In [9]:
SAVE_NUMPY_PATH = CURRENT_PATH + '/../data/numpy_results/'
# Original load_data from data.py
speakers = load_data(DATA_PATH, verbose=True, num_samples=80, ctc_encoding=True)

new speaker added: s13
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae2p.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaq9s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaezn.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak4n.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae1s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaq8n.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak7a.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak5s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae3a.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak6p.mpg
new spe

In [10]:
epochs = 100
x, y, label_len, input_len = speakers
print("training data shapes:", x.shape, y.shape)
x_train, x_test, y_train, y_test, label_len_train, label_len_test, \
input_len_train, input_len_test = train_test_split(x, y, label_len, input_len, test_size=0.2)

model = build_model(x.shape[1:], 28, max_string_len = 10)

history = train(model, x_train, y_train, label_len_train, input_len_train, epochs=epochs)

print("Saving model...")
model.save('model.h5')

('training data shapes:', (80, 16, 50, 100, 3), (80, 6))

after dense1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 16, 50, 100,  0                                            
__________________________________________________________________________________________________
padding1 (ZeroPadding3D)        (None, 16, 54, 104,  0           the_input[0][0]                  
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 16, 27, 52, 3 2432        padding1[0][0]                   
__________________________________________________________________________________________________
time_distributed_2 (TimeDistrib (None, 16, 13, 26, 3 0           time_distributed_1[0][0]         
______________________________________

Epoch 36/100
51/51 [==============================] - 2s 39ms/step - loss: 15.1430 - val_loss: 17.4848
Epoch 37/100
51/51 [==============================] - 2s 39ms/step - loss: 15.1448 - val_loss: 17.4540
Epoch 38/100
51/51 [==============================] - 2s 39ms/step - loss: 15.1063 - val_loss: 17.4264
Epoch 39/100
51/51 [==============================] - 2s 39ms/step - loss: 15.0708 - val_loss: 17.4018
Epoch 40/100
51/51 [==============================] - 2s 39ms/step - loss: 15.0303 - val_loss: 17.3798
Epoch 41/100
51/51 [==============================] - 2s 39ms/step - loss: 15.0064 - val_loss: 17.3610
Epoch 42/100
51/51 [==============================] - 2s 39ms/step - loss: 14.9974 - val_loss: 17.3454
Epoch 43/100
51/51 [==============================] - 2s 39ms/step - loss: 14.9695 - val_loss: 17.3325
Epoch 44/100
51/51 [==============================] - 2s 39ms/step - loss: 14.9716 - val_loss: 17.3217
Epoch 45/100
51/51 [==============================] - 2s 39ms/step - loss

In [63]:
# load_data2 takes the same inputs as load_data, but detect speakers and save corresponding numpy results

In [35]:
import os
from align import read_align
from video import read_video
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import re

# CURRENT_PATH = os.path.dirname(os.path.abspath(__file__))
CURRENT_PATH = '/home/ubuntu/assignments/machine-lip-reading/preprocessing'
DATA_PATH = CURRENT_PATH + '/../data'
PREDICTOR_PATH = CURRENT_PATH + '/shape_predictor_68_face_landmarks.dat'
SAVE_NUMPY_PATH = CURRENT_PATH + '/../data/numpy_results'


def text_to_labels(text):
    ret = []
    for char in text:
        if char >= 'a' and char <= 'z':
            ret.append(ord(char) - ord('a'))
        elif char == ' ':
            ret.append(26)
    return ret

def labels_to_text(labels):
# 26 is space, 27 is CTC blank char
    text = ''
    for c in labels:
        if c >= 0 and c < 26:
            text += chr(c + ord('a'))
        elif c == 26:
            text += ' '
    return text


def load_data2(datapath, verbose=False, num_samples=-1, ctc_encoding=False):
    oh = OneHotEncoder()
    le = LabelEncoder()

    counter = 0
    done = False

    max_len = 0
    max_word_len = 0

    word_len_list = []
    input_len_list = []

    x_raw = list()
    y_raw = list()
    
    pattern = re.compile("s[0-9]") 
    speakers = []
    x_raws = {}
    y_raws = {}
    word_len_lists = {}
    input_len_lists = {}
    
    for root, dirs, files in os.walk(datapath):
        check = root.split("/")[-1]
        match = pattern.findall(check)
        if (len(match) > 0):
            if check.index(match[0]) == 0 and check not in speakers:             
                speakers.append(check)
                x_raws[check] = []
                y_raws[check] = []
                word_len_lists[check] = []
                input_len_lists[check] = []
                if verbose:
                    print("new speaker added: " + check)            
        
        for name in files:
            if '.mpg' in name:
                if verbose is True:
                    print("reading: " + root + "/" + name)

                video = read_video(os.path.join(root, name), PREDICTOR_PATH)
                alignments = read_align(os.path.join(root, '../align/', name.split(".")[0] + ".align"))
                
                for start, stop, word in alignments:
                    if word == 'sil' or word == 'sp':
                        continue
                    
#                     if verbose is True:
#                         print(str(counter) + ": " + str(start) + "--" + str(stop) + ": " + word)
                    
                    _, d1, d2, d3 = video[start:stop].shape
                    
                    if (len(x_raw) > 0):
                        _, prev_d1, prev_d2, prev_d3 = x_raw[-1].shape
                        if (d1, d2, d3) != (prev_d1, prev_d2, prev_d3):
                            if verbose is True:
                                print("different size, skip")
                            continue
                    
                    x_raw.append(video[start:stop])
                    y_raw.append(word)
                    
                    x_raws[speakers[-1]].append(video[start:stop])
                    y_raws[speakers[-1]].append(word)
                    
                    max_word_len = max(max_word_len, len(word))
                    max_len = max(max_len, stop-start)
                    word_len_list.append(len(word))
                    input_len_list.append(stop-start)
                    
                    word_len_lists[speakers[-1]].append(len(word))
                    input_len_lists[speakers[-1]].append(stop-start)
                    
                    counter += 1
                    if counter == num_samples:
                        done = True
                        break
            
            if done:
                break
        
        if done:
            break
    
    if not ctc_encoding:
        y_raw = le.fit_transform(y_raw)
        y = oh.fit_transform(y_raw.reshape(-1, 1)).todense()


    for i in range(len(x_raw)):
        result = np.zeros((max_len, x_raw[i].shape[1], x_raw[i].shape[2], x_raw[i].shape[3]))
        result[:x_raw[i].shape[0], :x_raw[i].shape[1], :x_raw[i].shape[2], :x_raw[i].shape[3]] = x_raw[i]
        x_raw[i] = result
        
        
        if ctc_encoding:
            res = np.ones(max_word_len) * -1
            enc = np.array(text_to_labels(y_raw[i]))
            res[:enc.shape[0]] = enc
            y_raw[i] = res

    
    
    for speaker in speakers:
        
        for i in range(len(x_raws[speaker])):
            result = np.zeros((max_len, x_raws[speaker][i].shape[1], x_raws[speaker][i].shape[2], x_raws[speaker][i].shape[3]))
            result[:x_raws[speaker][i].shape[0], :x_raws[speaker][i].shape[1], :x_raws[speaker][i].shape[2], :x_raws[speaker][i].shape[3]] = x_raws[speaker][i]
            x_raws[speaker][i] = result


            if ctc_encoding:
                res = np.ones(max_word_len) * -1
                enc = np.array(text_to_labels(y_raws[speaker][i]))
                res[:enc.shape[0]] = enc
                y_raws[speaker][i] = res

        np_save = SAVE_NUMPY_PATH + "/" + speaker
#         x_raws[speaker] = np.stack(x_raws[speaker], axis=0)
        np.save(np_save + "_x2", x_raws[speaker]) 
#         if ctc_encoding:
#             y = np.stack(y_raw, axis=0)
        np.save(np_save + "_y2", y_raws[speaker])
        np.save(np_save + "_word_len_list2", np.array(word_len_lists[speaker]))
        np.save(np_save + "_input_len_list2", np.array(input_len_lists[speaker]))
        
    if ctc_encoding:
        y = np.stack(y_raw, axis=0)

    x = np.stack(x_raw, axis=0)
    return speakers

In [55]:
speakers = load_data2(DATA_PATH, verbose=True, num_samples=150, ctc_encoding=True)

new speaker added: s13
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae2p.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaq9s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaezn.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak4n.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae1s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaq8n.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak7a.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak5s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae3a.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak6p.mpg
new spe

In [56]:
# train by loading data saved in load_data2
for speaker in speakers:
    
    print("hey let's loading the numpy data for speaker " + speaker)
    
    print("reading x_raw: ")
    x_raw = np.load(SAVE_NUMPY_PATH + "/" + speaker + "_x2.npy")
    x_raw = np.stack(x_raw, axis=0)
    print(x_raw.shape)
    
    print("reading y_raw: ")
    y_raw = np.load(SAVE_NUMPY_PATH + "/" + speaker + "_y2.npy")
    print(y_raw.shape)
    
    print("reading word_len_list: ")
    word_len_list = np.load(SAVE_NUMPY_PATH + "/" + speaker + "_word_len_list2.npy")
    print(word_len_list.shape)
    
    print("reading input_len_list: ")
    input_len_list = np.load(SAVE_NUMPY_PATH + "/" + speaker + "_input_len_list2.npy")
    print(input_len_list.shape)
    
    print("training data shapes:", x_raw.shape, y_raw.shape)
    x_train, x_test, y_train, y_test, label_len_train, label_len_test, \
    input_len_train, input_len_test = train_test_split(x_raw, y_raw, word_len_list, input_len_list, test_size=0.2)

    model = build_model(x_raw.shape[1:], 28, max_string_len = 10)

    history = train(model, x_train, y_train, label_len_train, input_len_train, epochs=200)

    print("Saving model...")
    model.save('model_'+ speaker +".h5")

    print("Done.")
    

hey let's loading the numpy data for speaker s13
reading x_raw: 
(60, 16, 50, 100, 3)
reading y_raw: 
(60, 6)
reading word_len_list: 
(60,)
reading input_len_list: 
(60,)
('training data shapes:', (60, 16, 50, 100, 3), (60, 6))

after dense1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 16, 50, 100,  0                                            
__________________________________________________________________________________________________
padding1 (ZeroPadding3D)        (None, 16, 54, 104,  0           the_input[0][0]                  
__________________________________________________________________________________________________
time_distributed_15 (TimeDistri (None, 16, 27, 52, 3 2432        padding1[0][0]                   
_________________________________________________________________

Epoch 34/200
38/38 [==============================] - 1s 39ms/step - loss: 16.4708 - val_loss: 18.0158
Epoch 35/200
38/38 [==============================] - 2s 40ms/step - loss: 16.4081 - val_loss: 17.9732
Epoch 36/200
38/38 [==============================] - 2s 40ms/step - loss: 16.4147 - val_loss: 17.9359
Epoch 37/200
38/38 [==============================] - 1s 39ms/step - loss: 16.2970 - val_loss: 17.9029
Epoch 38/200
38/38 [==============================] - 2s 40ms/step - loss: 16.2560 - val_loss: 17.8741
Epoch 39/200
38/38 [==============================] - 2s 40ms/step - loss: 16.2529 - val_loss: 17.8475
Epoch 40/200
38/38 [==============================] - 2s 40ms/step - loss: 16.1346 - val_loss: 17.8246
Epoch 41/200
38/38 [==============================] - 1s 39ms/step - loss: 16.1186 - val_loss: 17.8044
Epoch 42/200
38/38 [==============================] - 1s 39ms/step - loss: 16.0225 - val_loss: 17.7863
Epoch 43/200
38/38 [==============================] - 2s 40ms/step - loss

Epoch 114/200
38/38 [==============================] - 1s 39ms/step - loss: 15.7018 - val_loss: 17.6194
Epoch 115/200
38/38 [==============================] - 2s 40ms/step - loss: 15.7011 - val_loss: 17.6192
Epoch 116/200
38/38 [==============================] - 1s 39ms/step - loss: 15.7014 - val_loss: 17.6190
Epoch 117/200
38/38 [==============================] - 2s 40ms/step - loss: 15.6957 - val_loss: 17.6188
Epoch 118/200
38/38 [==============================] - 2s 40ms/step - loss: 15.7003 - val_loss: 17.6186
Epoch 119/200
38/38 [==============================] - 1s 39ms/step - loss: 15.7029 - val_loss: 17.6184
Epoch 120/200
38/38 [==============================] - 1s 39ms/step - loss: 15.7113 - val_loss: 17.6182
Epoch 121/200
38/38 [==============================] - 2s 40ms/step - loss: 15.7104 - val_loss: 17.6180
Epoch 122/200
38/38 [==============================] - 2s 40ms/step - loss: 15.6995 - val_loss: 17.6178
Epoch 123/200
38/38 [==============================] - 2s 40ms/s

Epoch 193/200
38/38 [==============================] - 2s 40ms/step - loss: 15.6912 - val_loss: 17.6100
Epoch 194/200
38/38 [==============================] - 2s 40ms/step - loss: 15.6870 - val_loss: 17.6099
Epoch 195/200
38/38 [==============================] - 1s 39ms/step - loss: 15.6894 - val_loss: 17.6098
Epoch 196/200
38/38 [==============================] - 2s 40ms/step - loss: 15.6840 - val_loss: 17.6098
Epoch 197/200
38/38 [==============================] - 2s 40ms/step - loss: 15.6871 - val_loss: 17.6097
Epoch 198/200
38/38 [==============================] - 2s 40ms/step - loss: 15.6865 - val_loss: 17.6096
Epoch 199/200
38/38 [==============================] - 2s 40ms/step - loss: 15.6861 - val_loss: 17.6096
Epoch 200/200
38/38 [==============================] - 1s 39ms/step - loss: 15.6873 - val_loss: 17.6095
Saving model...
Done.
hey let's loading the numpy data for speaker s1
reading x_raw: 
(90, 16, 50, 100, 3)
reading y_raw: 
(90, 6)
reading word_len_list: 
(90,)
reading

Epoch 26/200
57/57 [==============================] - 2s 40ms/step - loss: 14.9229 - val_loss: 16.4666
Epoch 27/200
57/57 [==============================] - 2s 39ms/step - loss: 14.7639 - val_loss: 16.3980
Epoch 28/200
57/57 [==============================] - 2s 40ms/step - loss: 14.7221 - val_loss: 16.3184
Epoch 29/200
57/57 [==============================] - 2s 39ms/step - loss: 14.6446 - val_loss: 16.2288
Epoch 30/200
57/57 [==============================] - 2s 40ms/step - loss: 14.5824 - val_loss: 16.1370
Epoch 31/200
57/57 [==============================] - 2s 39ms/step - loss: 14.4896 - val_loss: 16.0443
Epoch 32/200
57/57 [==============================] - 2s 40ms/step - loss: 14.4725 - val_loss: 15.9511
Epoch 33/200
57/57 [==============================] - 2s 39ms/step - loss: 14.3164 - val_loss: 15.8621
Epoch 34/200
57/57 [==============================] - 2s 39ms/step - loss: 14.2394 - val_loss: 15.7772
Epoch 35/200
57/57 [==============================] - 2s 39ms/step - loss

Epoch 106/200
57/57 [==============================] - 2s 40ms/step - loss: 13.5668 - val_loss: 15.0174
Epoch 107/200
57/57 [==============================] - 2s 40ms/step - loss: 13.5741 - val_loss: 15.0171
Epoch 108/200
57/57 [==============================] - 2s 39ms/step - loss: 13.5741 - val_loss: 15.0167
Epoch 109/200
57/57 [==============================] - 2s 39ms/step - loss: 13.5693 - val_loss: 15.0164
Epoch 110/200
57/57 [==============================] - 2s 39ms/step - loss: 13.5681 - val_loss: 15.0160
Epoch 111/200
57/57 [==============================] - 2s 40ms/step - loss: 13.5642 - val_loss: 15.0157
Epoch 112/200
57/57 [==============================] - 2s 40ms/step - loss: 13.5628 - val_loss: 15.0154
Epoch 113/200
57/57 [==============================] - 2s 40ms/step - loss: 13.5642 - val_loss: 15.0151
Epoch 114/200
57/57 [==============================] - 2s 40ms/step - loss: 13.5738 - val_loss: 15.0149
Epoch 115/200
57/57 [==============================] - 2s 40ms/s

Epoch 185/200
57/57 [==============================] - 2s 39ms/step - loss: 13.5599 - val_loss: 15.0046
Epoch 186/200
57/57 [==============================] - 2s 40ms/step - loss: 13.5547 - val_loss: 15.0045
Epoch 187/200
57/57 [==============================] - 2s 39ms/step - loss: 13.5610 - val_loss: 15.0044
Epoch 188/200
57/57 [==============================] - 2s 40ms/step - loss: 13.5569 - val_loss: 15.0043
Epoch 189/200
57/57 [==============================] - 2s 39ms/step - loss: 13.5616 - val_loss: 15.0042
Epoch 190/200
57/57 [==============================] - 2s 39ms/step - loss: 13.5587 - val_loss: 15.0041
Epoch 191/200
57/57 [==============================] - 2s 39ms/step - loss: 13.5592 - val_loss: 15.0041
Epoch 192/200
57/57 [==============================] - 2s 39ms/step - loss: 13.5601 - val_loss: 15.0040
Epoch 193/200
57/57 [==============================] - 2s 39ms/step - loss: 13.5580 - val_loss: 15.0039
Epoch 194/200
57/57 [==============================] - 2s 39ms/s

In [ ]:
# check loading time for each speaker

In [59]:
start = time.time()

for speaker in speakers:
    
    print("hey let's loading the numpy data for speaker " + speaker)
    print("reading x_raw: ")
    x_raw = np.load(SAVE_NUMPY_PATH + "/" + speaker + "_x2.npy")
    x_raws = x_raw
    print(x_raw.shape)
    print("reading y_raw: ")
    y_raw = np.load(SAVE_NUMPY_PATH + "/" + speaker + "_y2.npy")
    y_raws[speaker] = y_raw
    print(y_raw.shape)
    print("reading word_len_list: ")
    word_len_list = np.load(SAVE_NUMPY_PATH + "/" + speaker + "_word_len_list2.npy")
    print(word_len_list.shape)
    print("reading input_len_list: ")
    input_len_list = np.load(SAVE_NUMPY_PATH + "/" + speaker + "_input_len_list2.npy")

    print(input_len_list.shape)

end = time.time()
print("load data took", end-start)

hey let's loading the numpy data for speaker s13
reading x_raw: 
(60, 16, 50, 100, 3)
reading y_raw: 
(60, 6)
reading word_len_list: 
(60,)
reading input_len_list: 
(60,)
hey let's loading the numpy data for speaker s1
reading x_raw: 
(90, 16, 50, 100, 3)
reading y_raw: 
(90, 6)
reading word_len_list: 
(90,)
reading input_len_list: 
(90,)
('load data took', 0.1183469295501709)


In [65]:
# train for specific speaker
def load_data_for_speaker(datapath, speaker_id, verbose=False, num_samples=-1, ctc_encoding=False):
    oh = OneHotEncoder()
    le = LabelEncoder()

    counter = 0
    done = False

    max_len = 0
    max_word_len = 0

    word_len_list = []
    input_len_list = []

    x_raw = list()
    y_raw = list()
    
    path = datapath + "/" + speaker_id
    
    for root, dirs, files in os.walk(path):       
        
        for name in files:
            if '.mpg' in name:
                if verbose is True:
                    print("reading: " + root + "/" + name)

                video = read_video(os.path.join(root, name), PREDICTOR_PATH)
                alignments = read_align(os.path.join(root, '../align/', name.split(".")[0] + ".align"))
                
                for start, stop, word in alignments:
                    if word == 'sil' or word == 'sp':
                        continue
                    _, d1, d2, d3 = video[start:stop].shape
                    
                    if (len(x_raw) > 0):
                        _, prev_d1, prev_d2, prev_d3 = x_raw[-1].shape
                        if (d1, d2, d3) != (prev_d1, prev_d2, prev_d3):
                            if verbose is True:
                                print("different size, skip")
                            continue
                    
                    x_raw.append(video[start:stop])
                    y_raw.append(word)
                    
                    max_word_len = max(max_word_len, len(word))
                    max_len = max(max_len, stop-start)
                    word_len_list.append(len(word))
                    input_len_list.append(stop-start)
                    
                    counter += 1
                    if counter == num_samples:
                        done = True
                        break
            
            if done:
                break
        
        if done:
            break
    
    if not ctc_encoding:
        y_raw = le.fit_transform(y_raw)
        y = oh.fit_transform(y_raw.reshape(-1, 1)).todense()


    for i in range(len(x_raw)):
        result = np.zeros((max_len, x_raw[i].shape[1], x_raw[i].shape[2], x_raw[i].shape[3]))
        result[:x_raw[i].shape[0], :x_raw[i].shape[1], :x_raw[i].shape[2], :x_raw[i].shape[3]] = x_raw[i]
        x_raw[i] = result
        
        
        if ctc_encoding:
            res = np.ones(max_word_len) * -1
            enc = np.array(text_to_labels(y_raw[i]))
            res[:enc.shape[0]] = enc
            y_raw[i] = res

    
    
    np_save = SAVE_NUMPY_PATH + "/" + speaker_id
    np.save(np_save + "_x", x_raw) 
    np.save(np_save + "_y", y_raw)
    np.save(np_save + "_word_len_list", np.array(word_len_list))
    np.save(np_save + "_input_len_list", np.array(input_len_list))
        
#     if ctc_encoding:
#         y = np.stack(y_raw, axis=0)

#     x = np.stack(x_raw, axis=0)
    return speaker_id

In [66]:
speakers = load_data_for_speaker(DATA_PATH, 's13', verbose=True, num_samples=150, ctc_encoding=True)

reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae2p.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaq9s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaezn.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak4n.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae1s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbaq8n.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak7a.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak5s.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbae3a.mpg
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s13/video/bbak6p.mpg


In [67]:
if speakers:
    start = time.time()
    print("hey let's loading the numpy data for speaker " + speakers)
    
    print("reading x_raw: ")
    x_raw = np.load(SAVE_NUMPY_PATH + "/" + speakers + "_x.npy")
    x_raw = np.stack(x_raw, axis=0)
    print(x_raw.shape)
    
    print("reading y_raw: ")
    y_raw = np.load(SAVE_NUMPY_PATH + "/" + speakers + "_y.npy")
    print(y_raw.shape)
    
    print("reading word_len_list: ")
    word_len_list = np.load(SAVE_NUMPY_PATH + "/" + speakers + "_word_len_list.npy")
    print(word_len_list.shape)
    
    print("reading input_len_list: ")
    input_len_list = np.load(SAVE_NUMPY_PATH + "/" + speakers + "_input_len_list.npy")
    print(input_len_list.shape)
    end = time.time()
    print("load data took", end-start)
    print("training data shapes:", x_raw.shape, y_raw.shape)
    x_train, x_test, y_train, y_test, label_len_train, label_len_test, \
    input_len_train, input_len_test = train_test_split(x_raw, y_raw, word_len_list, input_len_list, test_size=0.2)

    model = build_model(x_raw.shape[1:], 28, max_string_len = 10)

    history = train(model, x_train, y_train, label_len_train, input_len_train, epochs=200)

    print("Saving model...")
    model.save('model_'+ speakers +".h5")

    print("Done.")
    

hey let's loading the numpy data for speaker s13
reading x_raw: 
(60, 16, 50, 100, 3)
reading y_raw: 
(60, 6)
reading word_len_list: 
(60,)
reading input_len_list: 
(60,)
('load data took', 0.09496402740478516)
('training data shapes:', (60, 16, 50, 100, 3), (60, 6))

after dense1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 16, 50, 100,  0                                            
__________________________________________________________________________________________________
padding1 (ZeroPadding3D)        (None, 16, 54, 104,  0           the_input[0][0]                  
__________________________________________________________________________________________________
time_distributed_36 (TimeDistri (None, 16, 27, 52, 3 2432        padding1[0][0]                   
_________________________

Epoch 34/200
38/38 [==============================] - 2s 40ms/step - loss: 17.3532 - val_loss: 17.2170
Epoch 35/200
38/38 [==============================] - 2s 40ms/step - loss: 17.3003 - val_loss: 17.0985
Epoch 36/200
38/38 [==============================] - 2s 40ms/step - loss: 17.1073 - val_loss: 16.9859
Epoch 37/200
38/38 [==============================] - 2s 40ms/step - loss: 17.0492 - val_loss: 16.8691
Epoch 38/200
38/38 [==============================] - 2s 40ms/step - loss: 16.8877 - val_loss: 16.7517
Epoch 39/200
38/38 [==============================] - 1s 39ms/step - loss: 16.8653 - val_loss: 16.6440
Epoch 40/200
38/38 [==============================] - 2s 40ms/step - loss: 16.7892 - val_loss: 16.5449
Epoch 41/200
38/38 [==============================] - 2s 40ms/step - loss: 16.7285 - val_loss: 16.4492
Epoch 42/200
38/38 [==============================] - 2s 39ms/step - loss: 16.6263 - val_loss: 16.3649
Epoch 43/200
38/38 [==============================] - 1s 39ms/step - loss

Epoch 114/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9603 - val_loss: 15.6141
Epoch 115/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9572 - val_loss: 15.6136
Epoch 116/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9622 - val_loss: 15.6130
Epoch 117/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9588 - val_loss: 15.6125
Epoch 118/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9875 - val_loss: 15.6119
Epoch 119/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9623 - val_loss: 15.6114
Epoch 120/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9616 - val_loss: 15.6109
Epoch 121/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9588 - val_loss: 15.6104
Epoch 122/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9554 - val_loss: 15.6100
Epoch 123/200
38/38 [==============================] - 2s 40ms/s

Epoch 193/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9482 - val_loss: 15.5904
Epoch 194/200
38/38 [==============================] - 1s 39ms/step - loss: 15.9467 - val_loss: 15.5902
Epoch 195/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9527 - val_loss: 15.5901
Epoch 196/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9483 - val_loss: 15.5899
Epoch 197/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9477 - val_loss: 15.5898
Epoch 198/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9462 - val_loss: 15.5897
Epoch 199/200
38/38 [==============================] - 1s 39ms/step - loss: 15.9651 - val_loss: 15.5895
Epoch 200/200
38/38 [==============================] - 2s 40ms/step - loss: 15.9452 - val_loss: 15.5894
Saving model...
Done.
